In [ ]:
# Code for labeling our data

In [ ]:
# Open-ai key is stored in OPENAI-API-KEY

In [1]:
!pip install openai pandas

In [ ]:
# ensure required packages and compatible typing_extensions are installed
%pip install --upgrade openai "typing_extensions>=4.8.0" pandas

import os
import openai
import pandas as pd

# Try to get API key from Colab userdata if available, otherwise fall back to environment variables
api_key = None
try:
    from google.colab import userdata  # type: ignore
    api_key = userdata.get("OPENAI-API-KEY")
except Exception:
    # not running in Colab or userdata not available
    pass

if not api_key:
    api_key = os.environ.get("OPENAI_API_KEY") or os.environ.get("OPENAI-API-KEY")

if not api_key:
    # Fallback: use provided API key (note: storing keys in notebooks is insecure)
    api_key = "-insert-your-openai-api-key-here-"

# configure openai client
openai.api_key = api_key

# Try to load the dataset; support both Colab /content path and local path
csv_paths = ["/content/reddit_w_stopwords.csv", "reddit_w_stopwords.csv"]
df = None
for p in csv_paths:
    try:
        df = pd.read_csv(p)
        print(f"Loaded data from {p} (rows={len(df)})")
        break
    except FileNotFoundError:
        continue

if df is None:
    # create an empty dataframe placeholder to avoid errors in later cells
    print("Could not find reddit_w_stopwords.csv in expected locations. Creating empty dataframe.")
    df = pd.DataFrame()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 2.6.1
    Uninstalling openai-2.6.1:
      Successfully uninstalled openai-2.6.1
Note: you may need to restart the kernel to use updated packages.
Loaded data from reddit_w_stopwords.csv (rows=10225)


In [3]:
df.head()

,subreddit,post_type,post_title,post_body,post_score,post_upvotes,post_downvotes,comment,comment_score,post_date,comment_date
0,Liberal,hot,Bill Gates and climate change,Apparently I was too upset earlier when I post...,10,7,3,Bill Gates isn’t a scientific expert on the to...,11,2025-10-29 16:51:35,2025-10-29 17:23:38
1,Liberal,hot,Bill Gates and climate change,Apparently I was too upset earlier when I post...,10,7,3,"So, he hasn't given up on climate, he's just c...",4,2025-10-29 16:51:35,2025-10-29 17:43:16
2,Liberal,hot,Bill Gates and climate change,Apparently I was too upset earlier when I post...,10,7,3,Bill Gates is very likely on the Epstein list ...,10,2025-10-29 16:51:35,2025-10-29 17:05:24
3,Liberal,hot,Bill Gates and climate change,Apparently I was too upset earlier when I post...,10,7,3,"OP, you have linked to an opinion article abou...",3,2025-10-29 16:51:35,2025-10-29 17:43:20
4,Liberal,hot,Bill Gates and climate change,Apparently I was too upset earlier when I post...,10,7,3,I’ve taken runny shits with more integrity tha...,5,2025-10-29 16:51:35,2025-10-29 17:04:56


In [4]:
def label_row_with_gpt(row):
  prompt = f"""
You are helping me label this row of data containing comments and posts from Reddit. Below is the description of each column in the row for context:
subreddit: the subreddit this comment was taken from
post_type: whether this post is hot, controversial, top, etc.
post_title: the title of the post
post_body: the body of the post
post_score: post upvotes + post downvotes (shows engagement on post)
post_upvotes: number of upvotes the post received
post_downvotes: number of downvotes the post received
comment: the main comment body
comment_score: upvotes + downvotes on the comment (shows engagement on the comment)
post_date: date the post was created
comment_date: date the comment was made
Each row represents a single comment.
Your task is to analyze the political leaning expressed in the comment, using the text content in the following columns:
comment (primary signal): {row['comment']}
post_title (context for the topic): {row['post_title']}
post_body (additional context and framing): {row['post_body']}
subreddit (weak proxy for either left or right): {row['subreddit']}
Assign a numeric score between -1 and 1 representing the ideological leaning:
-1 = extremely right-leaning
1 = extremely left-leaning
When determining the score:
Base it on the sentiment and stance expressed toward political topics, ideologies, figures, or issues.
Consider linguistic cues, framing, and opinions in both the post and the comment.
If a comment is apolitical, neutral, or unrelated to ideology, assign it a score of 0 (centrist).
Ensure consistent scoring across similar comments to maintain labeling reliability.
Here is a brief description of how the scores should be distributed based on political stance:
-1.00 to -0.75 → highly right-leaning
-0.75 to -0.25 → right-leaning
-0.25 to 0.25 → centrist
0.25 to 0.75 → left-leaning
0.75 to 1.00 → highly left-leaning
Your return value should only be a value between -1 and 1, no other words or values.
  """
  client = openai.OpenAI(api_key=api_key)
  response = client.chat.completions.create(
      model="gpt-5-mini",
      messages=[
          {"role": "user", "content": prompt}
      ]
  )
  return response.choices[0].message.content.strip()

In [6]:
df1 = df.head(1000).copy()  # limit to first 1000 rows 
df1['ideology_score'] = df1.apply(label_row_with_gpt, axis=1)
df1.to_csv("labeled_data_1.csv", index=False)
df1.head() 

,subreddit,post_type,post_title,post_body,post_score,post_upvotes,post_downvotes,comment,comment_score,post_date,comment_date,ideology_score
0,Liberal,hot,Bill Gates and climate change,Apparently I was too upset earlier when I post...,10,7,3,Bill Gates isn’t a scientific expert on the to...,11,2025-10-29 16:51:35,2025-10-29 17:23:38,0.4
1,Liberal,hot,Bill Gates and climate change,Apparently I was too upset earlier when I post...,10,7,3,"So, he hasn't given up on climate, he's just c...",4,2025-10-29 16:51:35,2025-10-29 17:43:16,0.6
2,Liberal,hot,Bill Gates and climate change,Apparently I was too upset earlier when I post...,10,7,3,Bill Gates is very likely on the Epstein list ...,10,2025-10-29 16:51:35,2025-10-29 17:05:24,0.7
3,Liberal,hot,Bill Gates and climate change,Apparently I was too upset earlier when I post...,10,7,3,"OP, you have linked to an opinion article abou...",3,2025-10-29 16:51:35,2025-10-29 17:43:20,0.6
4,Liberal,hot,Bill Gates and climate change,Apparently I was too upset earlier when I post...,10,7,3,I’ve taken runny shits with more integrity tha...,5,2025-10-29 16:51:35,2025-10-29 17:04:56,0.7


In [9]:
df2 = df.iloc[1000:2000].copy()  # next 1000 rows
df2['ideology_score'] = df2.apply(label_row_with_gpt, axis=1)
df2.to_csv("labeled_data_2.csv", index=False)
df2.head()

,subreddit,post_type,post_title,post_body,post_score,post_upvotes,post_downvotes,comment,comment_score,post_date,comment_date,ideology_score
1000,Liberal,controversial,What is liberal view on illegal immigration? I...,I find myself struggling to be upset that Trum...,100,67,33,Republicans don't want to solve illegal immigr...,51,2025-01-18 21:56:57,2025-01-18 22:26:58,0.6
1001,Liberal,controversial,What is liberal view on illegal immigration? I...,I find myself struggling to be upset that Trum...,100,67,33,90% of the hysteria around immigration is a bu...,38,2025-01-18 21:56:57,2025-01-18 22:50:21,0.9
1002,Liberal,controversial,What is liberal view on illegal immigration? I...,I find myself struggling to be upset that Trum...,100,67,33,I don't like that they came here through illeg...,18,2025-01-18 21:56:57,2025-01-18 23:26:25,0.15
1003,Liberal,controversial,What is liberal view on illegal immigration? I...,I find myself struggling to be upset that Trum...,100,67,33,The way to stop all the illegal immigration is...,9,2025-01-18 21:56:57,2025-01-19 00:25:19,0.9
1004,Liberal,controversial,What is liberal view on illegal immigration? I...,I find myself struggling to be upset that Trum...,100,67,33,Illegal immigration is a problem but there was...,35,2025-01-18 21:56:57,2025-01-18 22:04:30,0.6


In [7]:
df3 = df.iloc[2000:3000].copy()  # next 1000 rows
df3['ideology_score'] = df3.apply(label_row_with_gpt, axis=1)
df3.to_csv("labeled_data_3.csv", index=False)
df3.head() 

,subreddit,post_type,post_title,post_body,post_score,post_upvotes,post_downvotes,comment,comment_score,post_date,comment_date,ideology_score
2000,democrats,top,I have faith in my country,NaN,17975,17076,899,This takes balls. Hats off to those who made a...,328,2025-04-05 22:02:06,2025-04-05 22:25:01,0
2001,democrats,top,I have faith in my country,NaN,17975,17076,899,https://preview.redd.it/yim3qsjam3te1.jpeg?wid...,133,2025-04-05 22:02:06,2025-04-05 23:12:07,0
2002,democrats,top,I have faith in my country,NaN,17975,17076,899,Ours was SO BIG. Inspiring.,127,2025-04-05 22:02:06,2025-04-05 22:22:59,0
2003,democrats,top,I have faith in my country,NaN,17975,17076,899,Where there are no dots… i guess it is because...,118,2025-04-05 22:02:06,2025-04-05 22:21:21,0.6
2004,democrats,top,I have faith in my country,NaN,17975,17076,899,https://preview.redd.it/cdkl6h0rw3te1.jpeg?wid...,75,2025-04-05 22:02:06,2025-04-06 00:10:19,0


In [10]:
df4 = df.iloc[3000:4000].copy()  # next 1000 rows
df4['ideology_score'] = df4.apply(label_row_with_gpt, axis=1)
df4.to_csv("labeled_data_4.csv", index=False)
df4.head()

,subreddit,post_type,post_title,post_body,post_score,post_upvotes,post_downvotes,comment,comment_score,post_date,comment_date,ideology_score
3000,conservatives,top,Liz Cheney personally heard testimony that Tru...,NaN,440,400,40,Can't wait until AG Bondi appoints Special Cou...,17,2024-11-22 14:32:17,2024-11-22 18:34:10,-0.6
3001,conservatives,top,Ilhan Omar Facing Deportation Effort,NaN,430,365,65,"If she is here illegally, prove it and deport ...",189,2025-02-15 20:37:34,2025-02-15 21:17:54,-0.6
3002,conservatives,top,Ilhan Omar Facing Deportation Effort,NaN,430,365,65,I just don't get why people still vote for her...,34,2025-02-15 20:37:34,2025-02-15 22:52:50,-0.9
3003,conservatives,top,Ilhan Omar Facing Deportation Effort,NaN,430,365,65,"She hates America. So, send her home.",78,2025-02-15 20:37:34,2025-02-15 20:44:24,-0.95
3004,conservatives,top,Ilhan Omar Facing Deportation Effort,NaN,430,365,65,https://preview.redd.it/wlaare09ldje1.jpeg?wid...,40,2025-02-15 20:37:34,2025-02-15 22:02:20,0


In [13]:
df5 = df.iloc[4000:5000].copy()  # next 1000 rows
df5['ideology_score'] = df5.apply(label_row_with_gpt, axis=1)
df5.to_csv("labeled_data_5.csv", index=False)
df5.head()


,subreddit,post_type,post_title,post_body,post_score,post_upvotes,post_downvotes,comment,comment_score,post_date,comment_date,ideology_score
4000,PoliticalDiscussion,top,Republican Senate leader Mitch McConnell ackno...,Link to his words on it:\n\n* https://x.com/ha...,1526,1434,92,I think it’s too little too late. The border b...,33,2024-11-02 21:44:41,2024-11-02 22:23:44,0.6
4001,PoliticalDiscussion,top,Republican Senate leader Mitch McConnell ackno...,Link to his words on it:\n\n* https://x.com/ha...,1526,1434,92,Donald Trump could dissolve the border patrol ...,65,2024-11-02 21:44:41,2024-11-02 22:27:14,0.7
4002,PoliticalDiscussion,top,Republican Senate leader Mitch McConnell ackno...,Link to his words on it:\n\n* https://x.com/ha...,1526,1434,92,Guys…. None of this will impact him. I honestl...,17,2024-11-02 21:44:41,2024-11-02 22:59:55,0.9
4003,PoliticalDiscussion,top,Republican Senate leader Mitch McConnell ackno...,Link to his words on it:\n\n* https://x.com/ha...,1526,1434,92,"He didn't have to, because we all witnessed Tr...",11,2024-11-02 21:44:41,2024-11-02 22:46:58,0.9
4004,PoliticalDiscussion,top,Republican Senate leader Mitch McConnell ackno...,Link to his words on it:\n\n* https://x.com/ha...,1526,1434,92,No impact what so ever. Democrats were vocal a...,32,2024-11-02 21:44:41,2024-11-02 22:22:57,0.6


In [6]:
df6 = df.iloc[5000:6000].copy()  # next 1000 rows
df6['ideology_score'] = df6.apply(label_row_with_gpt, axis=1)
df6.to_csv("labeled_data_6.csv", index=False)
df6.head()


,subreddit,post_type,post_title,post_body,post_score,post_upvotes,post_downvotes,comment,comment_score,post_date,comment_date,ideology_score
5000,Politics,hot,"Trump, 79, Grudgingly Mumbles that He Really C...",NaN,1137,1080,57,I think that if he was confident he’d be alive...,1,2025-10-29 14:28:29,2025-10-29 14:42:40,0.5
5001,Politics,hot,"Trump, 79, Grudgingly Mumbles that He Really C...",NaN,1137,1080,57,He’d lose to Obama and his ego couldn’t stand ...,1,2025-10-29 14:28:29,2025-10-29 15:06:46,0.6
5002,Politics,hot,"Trump, 79, Grudgingly Mumbles that He Really C...",NaN,1137,1080,57,We need to stop giving ridiculous crap like th...,1,2025-10-29 14:28:29,2025-10-29 14:48:13,0
5003,Politics,hot,"Trump, 79, Grudgingly Mumbles that He Really C...",NaN,1137,1080,57,Why aren’t the articles about the absurdity in...,1,2025-10-29 14:28:29,2025-10-29 14:52:40,0.6
5004,Politics,hot,"Trump, 79, Grudgingly Mumbles that He Really C...",NaN,1137,1080,57,It’s too early to be talking about that. He ha...,1,2025-10-29 14:28:29,2025-10-29 14:54:16,0.5


In [7]:
df7 = df.iloc[6000:7000].copy()  # next 1000 rows
df7['ideology_score'] = df7.apply(label_row_with_gpt, axis=1)
df7.to_csv("labeled_data_7.csv", index=False)
df7.head()


,subreddit,post_type,post_title,post_body,post_score,post_upvotes,post_downvotes,comment,comment_score,post_date,comment_date,ideology_score
6000,Politics,controversial,Ballot Box Fire Investigators Find Devices Wit...,NaN,1,0,1,"\nAs a reminder, this subreddit [is for civil ...",1,2024-10-29 23:57:15,2024-10-29 23:57:15,0
6001,Politics,controversial,Ballot Box Fire Investigators Find Devices Wit...,NaN,1,0,1,This has Roger Stone written all over it.,66,2024-10-29 23:57:15,2024-10-29 23:59:02,0.5
6002,Politics,controversial,Ballot Box Fire Investigators Find Devices Wit...,NaN,1,0,1,MAGAts trying to sow division just like Russia...,58,2024-10-29 23:57:15,2024-10-29 23:58:51,0.85
6003,Politics,controversial,Ballot Box Fire Investigators Find Devices Wit...,NaN,1,0,1,I remember those boogaloo boys who shot the po...,23,2024-10-29 23:57:15,2024-10-30 00:02:23,0.15
6004,Politics,controversial,Ballot Box Fire Investigators Find Devices Wit...,NaN,1,0,1,"Before we dive into conspiracy theories, why d...",6,2024-10-29 23:57:15,2024-10-30 16:04:18,0


In [6]:
df8 = df.iloc[7000:8000].copy()  # next 1000 rows
df8['ideology_score'] = df8.apply(label_row_with_gpt, axis=1)
df8.to_csv("labeled_data_8.csv", index=False)
df8.head()

,subreddit,post_type,post_title,post_body,post_score,post_upvotes,post_downvotes,comment,comment_score,post_date,comment_date,ideology_score
7000,Ask_Politics,controversial,Did Trump really cause free Tuition wave in un...,"Hey guys, so I had recently talked with a Trum...",0,0,0,And it looks like MIT in particular has expand...,1,2024-11-27 06:45:09,2024-12-06 16:28:50,0
7001,Ask_Politics,controversial,Did Trump really cause free Tuition wave in un...,"Hey guys, so I had recently talked with a Trum...",0,0,0,"Weird, it was up 6 days ago.\n\nEdit: Here’s t...",1,2024-11-27 06:45:09,2024-12-04 02:53:51,0
7002,Ask_Politics,controversial,Why do people hate Christianity/Catholicism?,\nI have a hard time understanding why so many...,0,0,0,Welcome to /r/ask_politics. Our goal here is ...,1,2024-11-08 09:43:22,2024-11-08 09:43:22,0
7003,Ask_Politics,controversial,Why do people hate Christianity/Catholicism?,\nI have a hard time understanding why so many...,0,0,0,People don't usually hate Christianity specifi...,11,2024-11-08 09:43:22,2024-11-08 17:53:46,0
7004,Ask_Politics,controversial,Why do people hate Christianity/Catholicism?,\nI have a hard time understanding why so many...,0,0,0,dinosaurs knee cows capable axiomatic seed sno...,3,2024-11-08 09:43:22,2024-11-08 17:58:14,0


In [7]:
df9 = df.iloc[8000:9000].copy()  # next 1000 rows
df9['ideology_score'] = df9.apply(label_row_with_gpt, axis=1)
df9.to_csv("labeled_data_9.csv", index=False)
df9.head()


,subreddit,post_type,post_title,post_body,post_score,post_upvotes,post_downvotes,comment,comment_score,post_date,comment_date,ideology_score
8000,Conservative,hot,Five Republican Senators Join Democrats to Blo...,NaN,315,252,63,Rand is a shadow of his father,1,2025-10-29 02:19:39,2025-10-29 05:21:45,0
8001,Conservative,hot,"Was unhappy when the Dems flipped my district,...",NaN,17,15,2,That’s what we need to be doing is asking this...,1,2025-10-29 18:24:22,2025-10-29 19:10:59,-0.6
8002,Conservative,hot,Vance calls Rubio ‘my best friend in the admin...,NaN,44,39,5,I am curious if they will agree to a Vance/Rub...,1,2025-10-29 12:51:23,2025-10-29 13:09:22,-0.9
8003,Conservative,hot,Vance calls Rubio ‘my best friend in the admin...,NaN,44,39,5,You're selling Buttigieg short. He has been a ...,1,2025-10-29 12:51:23,2025-10-29 18:25:16,-0.5
8004,Conservative,hot,Vance calls Rubio ‘my best friend in the admin...,NaN,44,39,5,That last sentence. Republicans have a way of ...,1,2025-10-29 12:51:23,2025-10-29 14:33:03,0.5


In [8]:
# Final chunk all rows till end
df10 = df.iloc[9000:len(df)].copy()
df10['ideology_score'] = df10.apply(label_row_with_gpt, axis=1)
df10.to_csv("labeled_data_10.csv", index=False)
df10.head()

,subreddit,post_type,post_title,post_body,post_score,post_upvotes,post_downvotes,comment,comment_score,post_date,comment_date,ideology_score
9000,Conservative,controversial,New Yorkers celebrate assassination of UnitedH...,https://nypost.com/2024/12/07/us-news/new-york...,0,0,0,I get that people are upset about the state of...,8,2024-12-07 23:36:46,2024-12-08 05:51:33,-0.9
9001,Conservative,controversial,New Yorkers celebrate assassination of UnitedH...,https://nypost.com/2024/12/07/us-news/new-york...,0,0,0,Health insurance practices are scummy. They do...,4,2024-12-07 23:36:46,2024-12-08 03:45:13,-0.6
9002,Conservative,controversial,A Hell of A Campaign,NaN,2268,1338,930,It's easy to forget so much craziness that hap...,740,2024-11-04 23:20:12,2024-11-04 23:50:00,0
9003,Conservative,controversial,A Hell of A Campaign,NaN,2268,1338,930,Photo #9 is the most iconic photo of my lifetime.,460,2024-11-04 23:20:12,2024-11-04 23:51:02,0
9004,Conservative,controversial,A Hell of A Campaign,NaN,2268,1338,930,I truly believe this will go down as one of th...,174,2024-11-04 23:20:12,2024-11-05 01:00:16,-1.0


In [10]:
# combine labeled_data_1.csv, ..., labeled_data_10.csv into a single dataframe
df_list = []
for i in range(1, 11):
    df_chunk = pd.read_csv(f"labeled_data_{i}.csv")
    df_list.append(df_chunk)
combined_df = pd.concat(df_list, ignore_index=True)
combined_df.head()

,subreddit,post_type,post_title,post_body,post_score,post_upvotes,post_downvotes,comment,comment_score,post_date,comment_date,ideology_score
0,Liberal,hot,Bill Gates and climate change,Apparently I was too upset earlier when I post...,10,7,3,Bill Gates isn’t a scientific expert on the to...,11,2025-10-29 16:51:35,2025-10-29 17:23:38,0.4
1,Liberal,hot,Bill Gates and climate change,Apparently I was too upset earlier when I post...,10,7,3,"So, he hasn't given up on climate, he's just c...",4,2025-10-29 16:51:35,2025-10-29 17:43:16,0.6
2,Liberal,hot,Bill Gates and climate change,Apparently I was too upset earlier when I post...,10,7,3,Bill Gates is very likely on the Epstein list ...,10,2025-10-29 16:51:35,2025-10-29 17:05:24,0.7
3,Liberal,hot,Bill Gates and climate change,Apparently I was too upset earlier when I post...,10,7,3,"OP, you have linked to an opinion article abou...",3,2025-10-29 16:51:35,2025-10-29 17:43:20,0.6
4,Liberal,hot,Bill Gates and climate change,Apparently I was too upset earlier when I post...,10,7,3,I’ve taken runny shits with more integrity tha...,5,2025-10-29 16:51:35,2025-10-29 17:04:56,0.7


In [15]:
# make a new column ideology based on ideology_score
# -1.00 to -0.75 → highly right-leaning
# -0.75 to -0.25 → right-leaning
# -0.25 to 0.25 → centrist
# 0.25 to 0.75 → left-leaning
# 0.75 to 1.00 → highly left-leaning
def ideology_label(score):
    try:
        score = float(score)
    except ValueError:
        return "unknown"
    if -1.00 <= score < -0.75:
        return "highly right-leaning"
    elif -0.75 <= score < -0.25:
        return "right-leaning"
    elif -0.25 <= score <= 0.25:
        return "centrist"
    elif 0.25 < score <= 0.75:
        return "left-leaning"
    elif 0.75 < score <= 1.00:
        return "highly left-leaning"
    else:
        return "unknown"
combined_df['ideology'] = combined_df['ideology_score'].apply(ideology_label)
combined_df.head()
print(combined_df.shape)

(10225, 13)


In [16]:
combined_df.head()

,subreddit,post_type,post_title,post_body,post_score,post_upvotes,post_downvotes,comment,comment_score,post_date,comment_date,ideology_score,ideology
0,Liberal,hot,Bill Gates and climate change,Apparently I was too upset earlier when I post...,10,7,3,Bill Gates isn’t a scientific expert on the to...,11,2025-10-29 16:51:35,2025-10-29 17:23:38,0.4,left-leaning
1,Liberal,hot,Bill Gates and climate change,Apparently I was too upset earlier when I post...,10,7,3,"So, he hasn't given up on climate, he's just c...",4,2025-10-29 16:51:35,2025-10-29 17:43:16,0.6,left-leaning
2,Liberal,hot,Bill Gates and climate change,Apparently I was too upset earlier when I post...,10,7,3,Bill Gates is very likely on the Epstein list ...,10,2025-10-29 16:51:35,2025-10-29 17:05:24,0.7,left-leaning
3,Liberal,hot,Bill Gates and climate change,Apparently I was too upset earlier when I post...,10,7,3,"OP, you have linked to an opinion article abou...",3,2025-10-29 16:51:35,2025-10-29 17:43:20,0.6,left-leaning
4,Liberal,hot,Bill Gates and climate change,Apparently I was too upset earlier when I post...,10,7,3,I’ve taken runny shits with more integrity tha...,5,2025-10-29 16:51:35,2025-10-29 17:04:56,0.7,left-leaning


In [17]:
# save as combined_labeled_data.csv
combined_df.to_csv("combined_labeled_data.csv", index=False)